# PES University, Bangalore
Established under Karnataka Act No. 16 of 2013

## UE23AM343AB1 - Advanced Data Analytics

Designed by Diya M Hande

### Team
-TEAM MEMBERS


        -Sai Hemanth Medicherla(SRN:PES2UG23AM088)


        -Sai Jaswanth Akula(SRN:PES2UG23AM089)


        -Udaya G(SRN:PES2UG23AM111)


        -Indira P(SRN:PES2UG23AM072)


### Student Details
- Name : **SAI HEMANTH MEDICHERLA**
- SRN : **PES2UG23AM088**

## Worksheet D - Case Study (India & USA)

### Overview
The Student Social Media & Relationships dataset contains anonymized records of students’ social‐media behaviors and related life outcomes. It spans multiple ages and academic levels, focusing on key dimensions such as usage intensity, platform preferences, and relationship dynamics. Each row represents one student’s survey response.

You are given two datasets:
- `usa_dataset.csv` → Larger dataset (training)
- `india_dataset.csv` → Smaller dataset (testing)

The USA dataset contains the following variables:
- Student_ID: Unique respondent identifier
- Age: Age in years
- Gender: “Male” or “Female”
- Academic_Level: High School / Undergraduate / Graduate
- Avg_Daily_Usage_Hours: Average hours per day on social media
- Most_Used_Platform:	Instagram, Facebook, TikTok, etc.
- Affects_Academic_Performance: Self‐reported impact on academics (Yes/No)
- Sleep_Hours_Per_Night: Average nightly sleep hours
- Mental_Health_Score: Self‐rated mental health (1 = poor to 10 = excellent)
- Relationship_Status: Single / In Relationship / Complicated
- Conflicts_Over_Social_Media:	Number of relationship conflicts due to social media
- Addicted_Score: Social Media Addiction Score (1 = low to 10 = high)

The India dataset contains the same variables, except "Addicted_Score" and "Relationship_Status" columns are missing. Your task is to predict the addicted score and relationship status for the Indians.

### Let’s dive in!

**NOTE:** Please read the questions provided in the case study ppt and follow the mind map provided for the case study if you need any hints.

### Step 1: Load and Inspect the Data
- Load both CSV files.
- View the first 5 rows of each.
- Check shape and data types.
- Figure out which variables are categorical and which ones are numerical.
- Perform some basic EDA


In [17]:
# code here
import pandas as pd #pandas for data manip
import numpy as np#numpy for ops
import matplotlib.pyplot as plt#for plotting results and all

train_df=pd.read_csv('usa_dataset.csv')#this is our train dataset
test_df=pd.read_csv('india_dataset.csv')#our test dataset

print(train_df.iloc[:,5])#prints first 5 rows
print(test_df.iloc[:,5])#prints first 5 rows
print(train_df.shape,test_df.shape)#prints shape of dataframes
print(train_df.info(),test_df.info())#prints info of dataframes
print(train_df.describe(),test_df.describe())#baisic stats




0      Instagram
1        YouTube
2       Facebook
3      Instagram
4       LinkedIn
         ...    
607       TikTok
608    Instagram
609       WeChat
610      Twitter
611     Facebook
Name: Most_Used_Platform, Length: 612, dtype: object
0       Twitter
1        TikTok
2        TikTok
3     Instagram
4     Instagram
        ...    
88      Twitter
89     Facebook
90      Twitter
91       TikTok
92      Twitter
Name: Most_Used_Platform, Length: 93, dtype: object
(612, 12) (93, 10)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 612 entries, 0 to 611
Data columns (total 12 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   Student_ID                    612 non-null    int64  
 1   Age                           612 non-null    int64  
 2   Gender                        612 non-null    object 
 3   Academic_Level                612 non-null    object 
 4   Avg_Daily_Usage_Hours         612 non-null    flo

### Step 2: Test Similarity of Distributions
Question: Are the two datasets from similar distributions?

Hints:
- If **numerical features**: Test for normality (Shapiro-Wilk).
- If normal → use **parametric tests** (e.g., T-test).
- If not normal → use **non-parametric tests** (Mann-Whitney U test).
- For more hints → check the case study mind map

Do this for at least two numerical features.

In [18]:
# code here
from scipy.stats import shapiro, ttest_ind, mannwhitneyu
#These 3 libraries have our tests that we need to perform 
stat1_train, p_train=shapiro(train_df['Mental_Health_Score'])
stat2_train,p_train2=shapiro(train_df['Avg_Daily_Usage_Hours'])
stat1_test, p_test=shapiro(test_df['Mental_Health_Score'])
stat2_test,p_test2=shapiro(test_df['Avg_Daily_Usage_Hours'])
# Check normality for Addicted Score
if p_train > 0.05 and p_test > 0.05:
    print("Addicted score is normally distributed")
    stat, p = ttest_ind(train_df['Mental_Health_Score'], test_df['Mental_Health_Score'])
    print("T-test result:", p, stat)
else:
    print("Addicted score is not normally distributed")
    stat, p = mannwhitneyu(train_df['Mental_Health_Score'], test_df['Mental_Health_Score'])
    print("Mann-Whitney U result:", p, stat)


# Check normality for Daily Usage
if p_train2 > 0.05 and p_test2 > 0.05:
    print("Daily usage is normally distributed")
    stat, p = ttest_ind(train_df['Avg_Daily_Usage_Hours'], test_df['Avg_Daily_Usage_Hours'])
    print("T-test result:", p, stat)
else:
    print("Daily usage is not normally distributed")
    stat, p = mannwhitneyu(train_df['Avg_Daily_Usage_Hours'], test_df['Avg_Daily_Usage_Hours'])
    print("Mann-Whitney U result:", p, stat)



Addicted score is not normally distributed
Mann-Whitney U result: 4.372527922988039e-20 44706.0
Daily usage is not normally distributed
Mann-Whitney U result: 4.56761229612571e-31 7254.0


### Step 3: Correlated Numerical Features
- Use **Pearson correlation** for numerical features.
- Remove one of the variables if correlation > threshold (ex: 0.8, only if required).

In [19]:
# code here
threshold=0.5#threshold value
to_drop=set()#set of columns to drop
corr_matrix=train_df.corr(method='pearson',numeric_only=True)#correlation matrix
for i in range(len(corr_matrix.columns)):
    for j in range(i+1,len(corr_matrix.columns)):
        corr_value=corr_matrix.iloc[i,j]
        if abs(corr_value)>threshold:
            to_drop.add(corr_matrix.columns[j])
print(to_drop)


{'Mental_Health_Score', 'Sleep_Hours_Per_Night', 'Conflicts_Over_Social_Media', 'Addicted_Score'}


### Step 4: Correlated Categorical Features
- Use **Chi-square test of independence** for categorical-categorical pairs.
- Build a contingency table for each pair.

In [26]:
# code here
from scipy.stats import chi2_contingency


cat_cols = train_df.select_dtypes(include='object').columns
most_related=[]

for i in range(len(cat_cols)):
    for j in range(i+1, len(cat_cols)):
        col1 = cat_cols[i]
        col2 = cat_cols[j]
        table = pd.crosstab(train_df[col1], train_df[col2])
        # Perform Chi-square test
        chi2, p, dof, expected = chi2_contingency(table)
        #print(f"{col1} vs {col2}: p-value={p}")
        if p < 0.05:
            most_related.append((col1, col2))
print(most_related)

[('Gender', 'Academic_Level'), ('Gender', 'Most_Used_Platform'), ('Academic_Level', 'Most_Used_Platform'), ('Academic_Level', 'Affects_Academic_Performance'), ('Academic_Level', 'Relationship_Status'), ('Most_Used_Platform', 'Affects_Academic_Performance'), ('Most_Used_Platform', 'Relationship_Status'), ('Affects_Academic_Performance', 'Relationship_Status')]


### Step 5: Significance of Numerical Features vs Label
- If label has 2 categories → Use **point biserial correlation** (not in the syllabus, but in the textbook).
- If label has >2 categories → Use **ANOVA** (if parametric) or **Kruskal-Wallis** (if non-parametric).


In [ ]:
from scipy.stats import pointbiserialr

# Identify numerical and binary categorical columns
numerical_cols = train_df.select_dtypes(include='number').columns
binary_cols = [col for col in train_df.columns if train_df[col].nunique() == 2 and train_df[col].dtype == 'object']

for num_col in numerical_cols:
    for bin_col in binary_cols:
        # Encode binary categorical column to numeric
        bin_encoded = train_df[bin_col].astype('category').cat.codes
        stat, p = pointbiserialr(train_df[num_col], bin_encoded)
        print(f"{num_col} vs {bin_col}: stat={stat}, p={p}")

Student_ID vs Gender: stat=-0.0010276362037785688, p=0.9797595616870317
Student_ID vs Affects_Academic_Performance: stat=0.011649942463995068, p=0.7736344065361943
Age vs Gender: stat=0.46962734515999954, p=6.76577086516803e-35
Age vs Affects_Academic_Performance: stat=-0.11755552501696737, p=0.003587532775806306
Avg_Daily_Usage_Hours vs Gender: stat=0.006425663610800214, p=0.8739534631537998
Avg_Daily_Usage_Hours vs Affects_Academic_Performance: stat=0.6671184125169828, p=4.810181978417886e-80
Sleep_Hours_Per_Night vs Gender: stat=-0.05522739367605889, p=0.172410689158084
Sleep_Hours_Per_Night vs Affects_Academic_Performance: stat=-0.5988605996393472, p=7.903463812010811e-61
Mental_Health_Score vs Gender: stat=-0.02479926296081962, p=0.5403129500996003
Mental_Health_Score vs Affects_Academic_Performance: stat=-0.8168778819338094, p=6.712462775935003e-148
Conflicts_Over_Social_Media vs Gender: stat=-0.037853845351101784, p=0.349853747221319
Conflicts_Over_Social_Media vs Affects_Academ

### Step 6: Select the Most Significant Features
- Drop features that are highly correlated or not statistically significant (only if required).
- Keep the best set for model training.


In [28]:
# code here

#already done in step 3

### Step 7: Train and Predict
- Train models on USA dataset (Hint: regression model for numerical data and classification model for categorical data).
- Predict missing variables in India dataset.
- Evaluate model performance.
- Create a submission.csv file that contains your predictions in the format:

Student_ID, Relationship_Status, Addicted_Score

In [ ]:
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestClassifier

# Features to encode (do NOT include target columns)
trans_ft = ['Gender', 'Most_Used_Platform']

# Drop only columns that exist (avoids KeyError if already dropped)
drop_cols = [col for col in ['Relationship_Status', 'Addicted_Score'] if col in train_df.columns]
X_train = train_df.drop(columns=drop_cols)
X_test = test_df.copy()  # test set does not have targets

# Build preprocessor for categorical and numerical features
preprocessor = ColumnTransformer(
    transformers=[
        ('cat', OneHotEncoder(drop='first'), trans_ft),
        ('num', StandardScaler(), X_train.select_dtypes(include='number').columns)
    ],
    remainder='drop'
)

# Transform features
x_train = preprocessor.fit_transform(X_train)
x_test = preprocessor.transform(X_test)

# Targets (check if columns exist)
y_relationship = train_df['Relationship_Status'] if 'Relationship_Status' in train_df.columns else None
y_addicted = train_df['Addicted_Score'] if 'Addicted_Score' in train_df.columns else None

# Train models only if targets exist
if y_addicted is not None:
    regressor = LinearRegression()
    regressor.fit(x_train, y_addicted)
    y_addicted_pred = regressor.predict(x_test)
else:
    y_addicted_pred = None

if y_relationship is not None:
    classifier = RandomForestClassifier(random_state=42)
    classifier.fit(x_train, y_relationship)
    y_relationship_pred = classifier.predict(x_test)
else:
    y_relationship_pred = None

# Print predictions (optional)
#print("Relationship_Status predictions:", y_relationship_pred)
#print("Addicted_Score predictions:", y_addicted_pred)


output_df = pd.DataFrame({
    'Student_ID': test_df['Student_ID'],
    'Relationship_Status': y_relationship_pred,
    'Addicted_Score': y_addicted_pred
})

# Save to CSV
output_df.to_csv('ind_pred.csv', index=False)
print('saved ')

saved 


In [ ]:
#to predict the accuracy of the model 


### Q&A
Answer the following based on your analysis:
1. Which numerical features are correlated?
2. Which categorical features are correlated?
3. Which features are most significant for the model?
4. Are USA and India distributions similar?

**Answer Here**

1) Mental_Health_Score', 'Sleep_Hours_Per_Night', 'Conflicts_Over_Social_Media', 'Addicted_Score'
2)'Gender', 'Academic_Level'), ('Gender', 'Most_Used_Platform'), ('Academic_Level', 'Most_Used_Platform'), ('Academic_Level', 'Affects_Academic_Performance'), ('Academic_Level', 'Relationship_Status'), ('Most_Used_Platform', 'Affects_Academic_Performance'), ('Most_Used_Platform', 'Relationship_Status'), ('Affects_Academic_Performance', 'Relationship_Status') are categorically coorelated by chi sq

3) All the highly correlated features are Mental_Health_Score
Sleep_Hours_Per_Night
Conflicts_Over_Social_Media
Avg_Daily_Usage_Hours
Gender
Most_Used_Platform
Academic_Level
Affects_Academic_Performance

technically we are supposed to drop addicted score but since it is a feature to be predicted, we don't
we can drop student ID.


4) Some of the features are similar yes, some of them have really similar distributions like we can see in step 2
